### https://docs.databricks.com/en/delta-live-tables/cdc.html#language-sql

In [0]:
import dlt
from pyspark.sql.functions import col, expr

@dlt.view
def cdc_view():
  return spark.readStream.option("skipChangeCommits","true").option("readChangeFeed", "true").table("hive_metastore.default.cust_table")

dlt.create_streaming_table("cust_table_scd1")

dlt.apply_changes(
  target = "cust_table_scd1",
  source = "cdc_view",
  keys = ["id"],
  sequence_by = col("_commit_timestamp"),
  apply_as_deletes = expr("_change_type = 'delete'"),
  apply_as_truncates = expr("_change_type = 'truncate'"),
  except_column_list = ["_change_type", "_commit_timestamp"],
  stored_as_scd_type = 1
)

In [0]:
dlt.create_streaming_table("cust_table_scd2")

dlt.apply_changes(
  target = "cust_table_scd2",
  source = "cdc_view",
  keys = ["id"],
  sequence_by = col("_commit_timestamp"),
  apply_as_deletes = expr("_change_type = 'delete'"),
  except_column_list = ["_change_type", "_commit_timestamp"],
  stored_as_scd_type = "2",
  track_history_except_column_list = ["email"]
)